<a href="https://colab.research.google.com/github/iwatake2222/3D_SpaceMouse/blob/master/utility/csv2tfrecord.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convert csv (Google cloud Auto ML Vision) format to TFRecord format
## Preparation

In [0]:
from google.colab import drive 
drive.mount('/content/drive')

%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)

## Functions

In [0]:
from PIL import Image
import os
import tensorflow as tf
from io import BytesIO
import csv
import argparse
import pathlib

def _bytes_feature(value):
	"""string / byte 型から byte_list を返す"""
	if isinstance(value, type(tf.constant(0))):
		value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
	if type(value) is list:
		return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))
	else:
		return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
	"""float / double 型から float_list を返す"""
	if type(value) is list:
		return tf.train.Feature(float_list=tf.train.FloatList(value=value))
	else:
		return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
	"""bool / enum / int / uint 型から Int64_list を返す"""
	if type(value) is list:
		return tf.train.Feature(int64_list=tf.train.Int64List(value=value))
	else:
		return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def conv_text2label(text):
	class_text2label = {
			"AsahiSuperDry": 1,
			"ClearAsahi": 2,
			"Hon_Kirin": 3,
			"Kinmugi": 4,
			"KIRIN_ICHI": 5,
			"Nodogoshi": 6,
			"PREMIUM_MALTS": 7,
			"SAPPORO": 8,
			"YEBISU": 9,
			}
	return class_text2label[text]

def write_features_to_record(image_filename, features_for_one_image):
	filename, ext = os.path.splitext(image_filename)
	with tf.io.TFRecordWriter(filename + ".tfrecord") as writer:
		image = Image.open(image_filename)
		image_string = open(image_filename, "rb").read()
		feature = {
				"image/encoded": _bytes_feature(image_string),
				"image/filename": _bytes_feature(image_filename.encode('utf-8')),
				"image/source_id": _bytes_feature(image_filename.encode('utf-8')),
				"image/format": _bytes_feature(ext[1:].encode('utf-8')),
				"image/height": _int64_feature(image.size[1]),
				"image/width": _int64_feature(image.size[0]),
				'image/depth': _int64_feature(len(image.split())),
				"image/object/bbox/xmax": _float_feature(list(map(float, [x[7] for x in features_for_one_image])) if len(features_for_one_image[0][3]) > 0 else []),
				"image/object/bbox/xmin": _float_feature(list(map(float, [x[3] for x in features_for_one_image])) if len(features_for_one_image[0][3]) > 0 else []),
				"image/object/bbox/ymax": _float_feature(list(map(float, [x[8] for x in features_for_one_image])) if len(features_for_one_image[0][3]) > 0 else []),
				"image/object/bbox/ymin": _float_feature(list(map(float, [x[4] for x in features_for_one_image])) if len(features_for_one_image[0][3]) > 0 else []),
				"image/object/class/label": _int64_feature(list(map(conv_text2label,[x[2] for x in features_for_one_image])) if len(features_for_one_image[0][3]) > 0 else []),
				"image/object/class/text": _bytes_feature([x[2].encode('utf-8') for x in features_for_one_image] if len(features_for_one_image[0][3]) > 0 else []),
#				"image/object/difficult": aaa,
#				"image/object/truncated": aaa,
#				"image/object/view": aaa,
				}
		tf_example = tf.train.Example(features=tf.train.Features(feature=feature))
		writer.write(tf_example.SerializeToString())

def convert_from_csv_to_frecord(path_to_csv):
	with open(path_to_csv) as f:
		reader = csv.reader(f)
		dataset_for_one_image = []
		for row in reader:
			if (len(dataset_for_one_image) > 0 and row[1] != dataset_for_one_image[-1][1]):
				image_filename = pathlib.Path(dataset_for_one_image[0][1]).name
				write_features_to_record(image_filename, dataset_for_one_image)
				dataset_for_one_image = []    # clear
			else:
				pass
			dataset_for_one_image.append(row)
	
		if (len(dataset_for_one_image) > 0):
			image_filename = pathlib.Path(dataset_for_one_image[0][1]).name
			write_features_to_record(image_filename, dataset_for_one_image)
   

## Execution
*IMPORTANT*

All image files must be placed in the same place as the current directory. (directory structure is ignored)


In [22]:
%cd "/content/drive/My Drive/temp/genarated_csv_dataset"
csv_filename = "annotation.csv"
convert_from_csv_to_frecord(csv_filename)
!ls

/content/drive/My Drive/temp/genarated_csv_dataset
20190922_201645.jpg  20190922_201645.tfrecord  annotation.csv
